# Project 8: Cross-Validation and Data Splits

## Goal
Master proper experimental design and validation strategies.

## Learning Objectives
- k-fold cross-validation
- Stratified splits for imbalanced data
- Hyperparameter tuning with CV
- Proper result reporting

In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
import sys
sys.path.append('/Users/mark/git/learning-ml-to-llm')

print("Ready to master cross-validation!")